In [1]:
import os
import pandas as pd
from pathlib import Path
import sys

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [2]:
# lista com os códigos CNES
def get_df_cnes():
  # base cnes, para pegar o código CNES da UBS
  path_folder_cnes = Path(os.getcwd()).resolve().parents[0]
  
  file_path = os.path.join(
    path_folder_cnes,
    'importa_csv_saude',
    'cadastro_estabelecimentos_cnes.csv')
  
  df_cnes = pd.read_csv(file_path, sep=";")
  df_cnes['NOME'] = df_cnes['NOME'].str.strip().str.upper()
  df_cnes['CO_CNES'] = df_cnes['CNES'].fillna(0).astype(int)
  df_cnes = df_cnes[['CO_CNES', 'NOME']]
  
  return df_cnes

In [3]:
df_cnes = get_df_cnes()
print(df_cnes.columns)

ufs = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
       'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
       'RS', 'SC', 'SE', 'SP', 'TO']

print(ufs)

for uf in ufs:
    file_path = os.path.join(f'postos_saude_brasil_{uf}.csv')
    df = pd.read_csv(file_path)

    tamanho = len(df)
    print(f'  Processando {str(tamanho).zfill(5)} registros')

    # inclui a coluna com o CEP
    df['address'] = df['address'].astype(str)
    df['cep'] = df['address'].str.extract(r'(\d{5}\-?\d{0,4})')

    # inclui a coluna com o código CNES
    df = df.merge(
      df_cnes, how='left', left_on='ESTABELECIMENTO', right_on='NOME')

    if 'CO_CNES_x' in df.columns:
      df['CO_CNES'] = df['CO_CNES_x']
      del df['CO_CNES_x']
    
    if 'CO_CNES_y' in df.columns:
      del df['CO_CNES_y']
    
    if 'CO_CNES' in df.columns:
      df['CO_CNES'] = df['CO_CNES'].fillna(0).astype(int)


    df = df[[
      'UF', 'MUNICIPIO', 'ESTABELECIMENTO', 'LOGRADOURO', 'NUMERO', 'BAIRRO',
       'ENDERECO_COMPLETO', 'latitude', 'longitude', 'address', 'point', 'cep',
       'CO_CNES']]

    df.to_csv(file_path, index=False)
    print('  Arquivo de saída atualizado com sucesso.', file_path)


Index(['CO_CNES', 'NOME'], dtype='object')
['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']
  Processando 00374 registros
  Arquivo de saída atualizado com sucesso. postos_saude_brasil_AC.csv
  Processando 04915 registros
  Arquivo de saída atualizado com sucesso. postos_saude_brasil_AL.csv
  Processando 02960 registros
  Arquivo de saída atualizado com sucesso. postos_saude_brasil_AM.csv
  Processando 00198 registros
  Arquivo de saída atualizado com sucesso. postos_saude_brasil_AP.csv
  Processando 29465 registros
  Arquivo de saída atualizado com sucesso. postos_saude_brasil_BA.csv
  Processando 12448 registros
  Arquivo de saída atualizado com sucesso. postos_saude_brasil_CE.csv
  Processando 00178 registros
  Arquivo de saída atualizado com sucesso. postos_saude_brasil_DF.csv
  Processando 05239 registros
  Arquivo de saída atualizado com sucesso. postos_saude_brasil_E